# Projet de Machine Learning

Notebook <b>R</b> avec les codes utilisés pour le rapport final.<br>
Auteurs : Juan AYALA, Jeong Hwan KO, Alice LALOUE, Aldo MELLADO AGUILAR.<br>
4A MA - Groupes A et B<br>
2020 - 2021

## Importation des librairies

In [ ]:
library(corrplot)
library(dplyr)
library(factoextra)
library(FactoMineR)
library(ggplot2)
library(glmnet)
library(RColorBrewer)
library(vcd)

In [ ]:
# Cette cellule permet de centrer les figures automatiquement
IRdisplay::display_html('
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
')

# Obtenir les données

In [ ]:
spotify_data <- read.table(file = "data/spotify-extr.txt", header = T, sep = " ")

# Description de l'ensemble du jeu de données

In [ ]:
head(spotify_data)

In [ ]:
str(spotify_data)

On n'a pas de valeurs manquantes donc on n'a pas besoin de les retravailler.

Les variables explicatives sont :
* `valence` : la positivité de la chanson, vaut 1 si la chanson est très joyeuse, 0 sinon ;
* `year` : année de sortie ;
* `acousticness` : mesure "l'acousticité" de la chanson ;
* `danceability` : mesure la "dançabilite" d'une chanson ;
* `duration` : durée d'une chanson en millisecondes ;
* `energy` : l'énergie de la chanson, vaut 1 si la chanson est très énergétique, 0 sinon ;
* `intrumentalness` : taux d'instrumentalisation, vaut 1 s'il n'y a aucune voix présente dans la chanson, 0 sinon ; 
* `key` : tonalité de la musique (ex : A=la), ne prend pas en compte la distinction majeur/mineur ;
* `liveness` : taux de prestation en live, vaut 1 si la chanson ne comporte que de la musique (sans sons à intérêts non-musicaux), 0 sinon ;
* `loudness` : intensité sonore de la chanson
* `mode` : variable binaire qui indique si la chanson commence par une progression d'accords majeure (1) ou non (0)
* `speechiness` : taux de vocaux dans la chanson, vaut 1 si la chanson comporte de la voix tout le long, 0 sinon ;
* `tempo` :  tempo de la chanson en beats par minute (bpm)

Notre objectif consiste à prédire la valeur de `pop.class` et de `popularity`, c'est-à-dire la popularité d'une chanson, soit comme un entier entre 0 et 100, soit comme une classe $A$, $B$, $C$ ou $D$.

In [ ]:
summary(spotify_data)

Dans notre jeu de données, les variables qualitatives sont :
* `pop.class`,
* `key`,
* `mode`.

Le reste des variables sont quantitatives.

On transforme les variables qualitatives en catégories pour mieux traiter les données.

In [ ]:
spotify_data$pop.class <- factor(spotify_data$pop.class, ordered = TRUE)
spotify_data$key <- factor(spotify_data$key)
spotify_data$mode <- factor(spotify_data$mode)

In [ ]:
pop.class <- spotify_data$pop.class
key <- spotify_data$key
mode <- spotify_data$mode

In [ ]:
rev(levels(pop.class))
levels(key)
levels(mode)

In [ ]:
sapply(spotify_data, class)

# Analyses uni et multidimensionnelles

## Variables qualitatives

On commence par analyser les variables qualitatives `pop.class`, `key` et `mode`.

In [ ]:
data.qual <- spotify_data[, c("key", "mode", "pop.class")]
head(data.qual)

<b>Classe de popularité</b> (variable à prédire)

Cette variable a été créée en amont de l'obtention des données.

In [ ]:
percentage <- as.data.frame(table(pop.class))
percentage$Freq <- percentage$Freq/sum(percentage$Freq)

pop.class.count <- data.frame(name = levels(pop.class), value = percentage$Freq)

options(repr.plot.width = 12, repr.plot.height = 8)

ggplot(data = pop.class.count, aes(x = name, y = value, fill = name)) +
    geom_bar(stat = "identity", show.legend = FALSE) +
    labs(x = "Classe", y = "Pourcentage d'occurences")

On voit qu'il y a une distribution uniforme des chansons par classe, sauf pour la classe `A`, qui comprend moins de 10% des chansons. Ceci risque de poser problème dans la suite en termes de prédiction.

<b>Clé</b>

In [ ]:
percentage <- as.data.frame(table(key))
percentage$Freq <- sort(percentage$Freq/sum(percentage$Freq), decreasing = F)

key.count <- data.frame(name = levels(key), value = percentage$Freq)

options(repr.plot.width = 15, repr.plot.height = 8)

ggplot(data = key.count, aes(x = name, y = value, fill = name)) +
    geom_bar(stat = "identity", show.legend = FALSE) +
    labs(x = "Clé", y = "% d'occurences") +
    geom_text(aes(label = paste(round(value, 3) * 100, "%"), hjust = -0.2)) +
    coord_flip()

In [ ]:
plotdata <- spotify_data %>%
    group_by(key) %>%
    summarize(n = n(), mean = mean(popularity), sd = sd(popularity)) %>%
    mutate(se = sd/sqrt(n)) %>%
    mutate(ic = se * qt((1 - 0.05)/2 + 0.5, n - 1))

ggplot(plotdata, aes(x = key, y = mean, fill = key)) +
    geom_bar(stat = "identity", show.legend = FALSE) +
    geom_linerange(aes(x = key, ymin = mean - ic, ymax = mean + ic)) +
    labs(x = "Clé", y = "Popularité moyenne")

Les variances de la popularité dans chacune des valeurs de `key` est petite donc nous n'avons pas besoin de transformer ces données.

In [ ]:
ggplot(data = spotify_data, aes(x = key, y = popularity, fill = key)) +
    geom_boxplot(show.legend = FALSE) +
    labs(x = "Clé", y = "Popularité")

De la même façon, la distribution de la popularité reste plutôt uniforme par clé : les boîtes ont une taille similaire et la médiane est au même niveau.

<b>Mode</b>

In [ ]:
percentage <- as.data.frame(table(mode))
percentage$Freq <- percentage$Freq/sum(percentage$Freq)

mode.count <- data.frame(name = levels(mode), value = percentage$Freq)

options(repr.plot.width = 12, repr.plot.height = 6)

ggplot(data = mode.count, aes(x = name, y = value, fill = name)) +
    geom_bar(stat = "identity", show.legend = FALSE) +
    labs(x = "Mode", y = "Pourcentage d'occurences")

La distribution de la variable `mode` est inégale : il y a 30% et 70% des chansons avec `mode` = 0 et `mode` = 1 respectivement.

In [ ]:
plotdata <- spotify_data %>%
    group_by(mode) %>%
    summarize(n = n(), mean = mean(popularity), sd = sd(popularity)) %>%
    mutate(se = sd/sqrt(n)) %>%
    mutate(ic = se * qt((1 - 0.05)/2 + 0.5, n - 1))

ggplot(plotdata, aes(x = mode, y = mean, fill = mode)) +
    geom_bar(stat = "identity", show.legend = FALSE) +
    geom_linerange(aes(x = mode, ymin = mean - ic, ymax = mean + ic)) +
    labs(x = "Mode", y = "Popularité moyenne")

Par contre, la popularité est similaire selon le mode.

In [ ]:
ggplot(data = spotify_data, aes(x = mode, y = popularity, fill = mode)) +
    geom_boxplot(show.legend = FALSE) +
    labs(x = "Mode", y = "Popularité")

On regroupe toutes les variables qualitatives en un barplot :

In [ ]:
plotdata <- spotify_data %>%
    group_by(key, mode) %>%
    summarize(n = n(), mean = mean(popularity), sd = sd(popularity)) %>%
    mutate(se = sd/sqrt(n)) %>%
    mutate(ic = se * qt((1 - 0.05)/2 + 0.5, n - 1))

options(repr.plot.width = 15, repr.plot.height = 6)

ggplot(data = plotdata, aes(x = key, y = mean, fill = key)) +
    geom_bar(position="dodge", stat="identity", show.legend = FALSE) +
    geom_linerange(aes(x = key, ymin = mean - ic, ymax = mean + ic)) +
    facet_wrap(~mode) +
    theme(strip.background = element_rect(colour="black", fill="white",
                                          size=1.5, linetype="solid"),
          strip.text.x = element_text(size=12, face="bold"))

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 6)

ggplot(data = spotify_data, aes(x = key, y = popularity, fill = key)) +
    geom_boxplot(show.legend = FALSE) +
    facet_wrap(~mode) +
    theme(strip.background = element_rect(colour="black", fill="white",
                                          size=1.5, linetype="solid"),
          strip.text.x = element_text(size=12, face="bold"))

## Variables quantitatives

On commence par visualiser la corrélation entre les variables quantitatives :

In [ ]:
data.quant <- spotify_data[, -c(8, 11, 15)]
colnames(data.quant)

In [ ]:
M <- cor(data.quant)
M

In [ ]:
library(ggcorrplot)

options(repr.plot.width = 12, repr.plot.height = 12)

M <- cor(data.quant)
ggcorrplot(M)

Ce graphique nous montre qu'il y a certaines variables qui ont une forte corrélation. Par exemple, il y a une forte corrélation négative entre les variables `energy` et `acousticness`. Cela a du sens vu que les chansons acoustiques sont plus tranquilles (moins énergiques) que celles qui ne sont pas acoustiques. De même, `energy` et `loudness` sont positivement corrélées, ce qui est attendu vu que les chansons bruyantes ont souvent plus d'énergie.
<br>
On voit aussi que plus une chanson est acoustique, moins elle est populaire, vu que les variables `acousticness` et `popularity` ont une forte corrélation négative.

In [ ]:
correlation <- sort(abs(M[, 10]), decreasing = T)
series <- as.data.frame(correlation)

print("Les variables les plus corrélées avec 'popularity' sont :")
for (row in rownames(series)) {
    if (series[row, 1] >= 0.2 && series[row, 1] < 1) {
        print(paste(row, round(series[row, 1], 2), "(abs)"))
    }
}

In [ ]:
for (i in 1:12) {
    vari <- data.quant[, i]
    nf <- layout(mat = matrix(c(1, 2), 2, 1, byrow = TRUE), height = c(1, 3))
    par(mar = c(5.1, 4.1, 1.1, 2.1))
    boxplot(vari, horizontal = TRUE, xaxt = "n", outline = F, col = "#5975A4")
    hist(vari, main = paste("Histogram of", colnames(data.quant)[i], sep = " "), 
        breaks = ifelse(i == 2, 100, 50), xlab = colnames(data.quant)[i], freq = F, 
        col = "#4FABBC")
    lines(density(vari))
}

Voici une étude plus approfondie de chaque variable quantitative :

<b>Acousticness</b>

In [ ]:
ax.data <- as.data.frame(spotify_data %>%
    group_by(acousticness) %>%
    mutate(popularity = mean(popularity)))

options(repr.plot.width = 16, repr.plot.height = 9)

ggplot(ax.data, aes(x = acousticness, y = popularity)) +
    geom_point(color = "blue", size = 4) +
    labs(y = "Popularité moyenne")

<b>Danceability</b>

In [ ]:
ax.data <- as.data.frame(spotify_data %>%
    group_by(danceability) %>%
    mutate(popularity = mean(popularity)))

options(repr.plot.width = 16, repr.plot.height = 9)

ggplot(ax.data, aes(x = danceability, y = popularity)) +
    geom_point(color = "blue", size = 4) +
    labs(y = "Popularité moyenne")

<b>Duration</b>

On convertit la durée des chansons en minutes pour en tirer plus d'informations.

In [ ]:
spotify_data$duration <- spotify_data$duration / 60000
summary(spotify_data$duration)

In [ ]:
ggplot(data = spotify_data, aes(x = duration)) +
    geom_histogram(color = "black", fill = "#7490C0", bins = 50) +
    labs(x = "duration (mins)")

On voit que la chanson la plus longue dans le jeu de données dure 45 minutes, donc on choisit de séparer les chansons longues de chansons courtes au seuil de 8 minutes pour mieux voir les durées.

In [ ]:
long.songs <- spotify_data[spotify_data$duration > 8, ]
short.songs <- spotify_data[spotify_data$duration <= 8, ]

In [ ]:
ggplot(data = short.songs, aes(x = duration)) +
    geom_histogram(color = "black", fill = "#7490C0", bins = 60) +
    labs(x = "duration (mins)")

In [ ]:
ggplot(data = long.songs, aes(x = duration)) +
    geom_histogram(color = "black", fill = "#7490C0", bins = 50) +
    labs(x = "duration (mins)")

In [ ]:
library(grid)
library(gridExtra)

ax1.data <- as.data.frame(short.songs %>%
    group_by(duration) %>%
    mutate(popularity = mean(popularity)))

ax2.data <- as.data.frame(long.songs %>%
    group_by(duration) %>%
    mutate(popularity = mean(popularity)))

p1 <- ggplot(ax1.data, aes(x = duration, y = popularity)) +
        geom_point(color = "blue", size = 4, alpha = .75) +
        labs(title = "Chansons courtes", y = "Popularité moyenne") +
        theme(plot.title = element_text(hjust = 0.5))

p2 <- ggplot(ax2.data, aes(x = duration, y = popularity)) +
        geom_point(color = "green", size = 4) +
        labs(title = "Chansons longues", y = "") +
        theme(plot.title = element_text(hjust = 0.5))

grid.arrange(p1, p2, nrow = 1)

<b>Energy</b>

In [ ]:
ax.data <- as.data.frame(short.songs %>%
    group_by(energy) %>%
    mutate(popularity = mean(popularity)))

ggplot(ax.data, aes(x = energy, y = popularity)) +
    geom_point(color = "blue", size = 4) +
    labs(y = "Popularité moyenne")

<b>Instrumentalness</b>

In [ ]:
summary(spotify_data$instrumentalness)

In [ ]:
dim(spotify_data[spotify_data$instrumentalness == 0, ])

Il y a 2806 chansons pour lesquelles `instrumentalness` vaut 0.

In [ ]:
ggplot(data = spotify_data, aes(x = instrumentalness)) +
    geom_histogram(color = "black", fill = "#7490C0", bins = 60)

In [ ]:
ax.data <- as.data.frame(spotify_data %>%
    group_by(instrumentalness) %>%
    mutate(popularity = mean(popularity)))

options(repr.plot.width = 16, repr.plot.height = 9)

ggplot(ax.data, aes(x = instrumentalness, y = popularity)) +
    geom_point(color = "blue", size = 4) +
    labs(y = "Popularité moyenne")

<b>Liveness</b>

In [ ]:
ax.data <- as.data.frame(short.songs %>%
    group_by(liveness) %>%
    mutate(popularity = mean(popularity)))

ggplot(ax.data, aes(x = liveness, y = popularity)) +
    geom_point(color = "blue", size = 4) +
    labs(y = "Popularité moyenne")

<b>Popularity</b> (variable à prédire)

In [ ]:
p1 <- ggplot(spotify_data, aes(x = popularity)) +
        geom_histogram(color = "black", fill = "#7490C0", bins = 50) +
        scale_x_continuous(breaks = seq(0, 100, 20)) +
        labs(x = "")

p2 <- ggplot(spotify_data[spotify_data$popularity > 0, ], aes(x = popularity)) +
        geom_histogram(color = "black", fill = "#7490C0", bins = 50) +
        scale_x_continuous(breaks = seq(0, 100, 20))

grid.arrange(p1, p2, nrow = 2, heights=c(20, 20),
             top = textGrob(
                 "Haut : Toutes les données\nBas : Popularité > 0",
                 gp = gpar(fontsize = 14)))

In [ ]:
dim(spotify_data[spotify_data$popularity == 0, ])
summary(spotify_data$popularity)

On voit qu'il y a un nombre important de chansons ayant 0 comme popularité. En effet ces chansons sont proches de l'extraction de la base des données et donc leur popularité n'avait pas encore été déterminée.

De plus, la valeur maximale de popularité est 93 et 50% des valeurs se trouvent entre 11 et 48. Ces éléments poseront des problèmes en termes de régression.

<b>Year</b>

In [ ]:
ax.data <- as.data.frame(spotify_data %>%
    group_by(year) %>%
    mutate(popularity = mean(popularity)))

options(repr.plot.width = 15, repr.plot.height = 5)

ggplot(data = ax.data, aes(x = year, y = popularity)) +
    geom_line(colour = "blue", size = 1) +
    scale_x_continuous(breaks = seq(1920, 2020, 5)) +
    labs(y = "Popularité moyenne")

<b>Tempo</b>

In [ ]:
library(WVPlots)

options(repr.plot.width = 10, repr.plot.height = 10)

ScatterHist(spotify_data, "tempo", "popularity", title = "Tempo vs. Popularity",
            minimal_labels = FALSE, smoothmethod = "none", binwidth_x = 4,
            binwidth_y = 4, point_alpha = 1, point_color = "#4C72B0",
            hist_color = "#7490C0")

In [ ]:
dim(spotify_data[spotify_data$tempo == 0, ])

On voit qu'il y a 13 chansons pour lesquelles `tempo` vaut 0 ce qui n'est pas possible.

In [ ]:
corrected.tempo <- spotify_data[spotify_data$tempo > 0, ]$tempo
summary(corrected.tempo)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 6)

ggplot(data = spotify_data, aes(x = tempo)) +
    geom_histogram(color = "black", fill = "#7490C0", bins = 200) +
    geom_text(x = 5, y = 55, label = "13 Outliers", size = 5) +
    geom_text(x = 135, y = 160, label = "Valeurs sans 0", size = 5) +
    annotate("text", x = 126, y = 40, label = "Médiane\ncorrigée\n114.55",
             size = 5, color = "green", fontface = 2) +
    geom_segment(aes(x = 114.55, y = 0, xend = 114.55, yend = 100),
                 linetype = "dashed", color = "green", size = 2) +
    geom_segment(aes(x = 35.37, y = 0, xend = 35.37, yend = 200),
                 linetype = "dashed", color = "orange", size = 2) +
    geom_segment(aes(x = 214.42, y = 0, xend = 214.42, yend = 200),
                 linetype = "dashed", color = "orange", size = 2) +
    geom_segment(aes(x = 0, y = 50, xend = 0, yend = 30),
                 arrow = arrow(length = unit(0.5, "cm"))) +
    geom_segment(aes(x = 35.37, y = 150, xend = 214.42, yend = 150),
                 linetype = "dashed", color = "red", size = 2)

On replace les valeurs où `tempo` = 0 par la médiane dans la colonne.

In [ ]:
median <- median(spotify_data[spotify_data$tempo > 0, ]$tempo)
median

dat <- replace(spotify_data$tempo, spotify_data$tempo == 0, median)
spotify_data$tempo <- dat
summary(spotify_data$tempo)

<b>Prétraitement de variables :</b>

On normalise la variable `danceability` vu sa ressemblance à une loi gaussienne puis on la supprime :

In [ ]:
spotify_data <- spotify_data %>%
    mutate(dance_norm = (danceability - mean(danceability)) / sd(danceability),
           danceability = NULL)

In [ ]:
options(repr.plot.width = 15, repr.plot.height = 9)

ggplot(data = spotify_data, aes(x = dance_norm)) +
    geom_histogram(color = "black", fill = "#7490C0", bins = 50)

In [ ]:
colnames(spotify_data)

In [ ]:
# options(repr.plot.width = 20, repr.plot.height = 20)
# pairs(~., data = data.quant, col = mode)

In [ ]:
# options(repr.plot.width = 20, repr.plot.height = 20)
# pairs(~., data = data.quant, col = key)

# ACP

In [ ]:
data.quant <- spotify_data[, -c(7, 10, 11, 14)]
colnames(data.quant)

In [ ]:
data.pca <- PCA(data.quant, scale.unit = T, graph = F, ncp = 11)

In [ ]:
eig.val <- get_eig(data.pca)

options(repr.plot.width = 16, repr.plot.height = 9)
par(mfrow = c(1, 2))

bp1 <- barplot(eig.val[1:10, 2], ylab = "Pourcentage de variance expliquée",
               ylim = c(0, 35), col = "#4682B4")
text(bp1, eig.val[1:10, 2] + 2, labels = paste(round(eig.val[, 2], 2), "%"))
lines(bp1, eig.val[1:10, 2])

bp2 <- barplot(eig.val[1:10, 3], ylab = "Variance partagée",
               ylim = c(0, 105), col = "#4682B4")
text(bp2, eig.val[1:10, 3] + 2, labels = paste(round(eig.val[, 3], 1), "%"))
lines(bp2, eig.val[1:10, 3])

In [ ]:
boxplot(data.pca$ind$coord)
abline(h = 0, col = "grey", lty = "dashed", lwd = 3)

In [ ]:
inds <- as.data.frame(data.pca$ind$coord)

options(repr.plot.width = 12, repr.plot.height = 9)

ggplot(inds, aes(inds[, 1], inds[, 2], colour = pop.class)) +
    geom_point() +
    labs(x = paste("Dim 1 (", round(eig.val[1, 2], 1), "%)"),
         y = paste("Dim 2 (", round(eig.val[2, 2], 1), "%)")) +
    scale_color_brewer(palette="Set1") +
    theme(legend.title = element_text(size = 16),
          legend.text = element_text(size = 13))

In [ ]:
fviz_pca_var(data.pca, col.var = "cos2",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             title = "")

In [ ]:
fviz_pca_var(data.pca, axes = c(1, 3), col.var = "cos2",
             gradient.cols = c("#00AFBB", "#E7B800", "#FC4E07"),
             title = "")

# Préparation des données

In [ ]:
spotify.pop.class <- spotify_data$pop.class
spotify.key <- spotify_data$key

In [ ]:
spotify.pop.class.encoded <- as.numeric(spotify.pop.class) - 1
spotify.pop.class.encoded[1:15]

In [ ]:
spotify.key.encoded <- as.numeric(spotify.key) - 1
spotify.key.encoded[1:15]

In [ ]:
spotify.pop.class[1:15]
spotify.key[1:15]

Pour la classe de popularité, on a la correspondance :
* 'A' -> 0
* 'B' -> 1
etc.

Puis pour la clé :
* 'A' -> 0
* 'Ab' -> 1<br>
$\dots$
* 'Gb' -> 11

In [ ]:
spotify_data$pop.class <- spotify.pop.class.encoded
spotify_data$key <- spotify.key.encoded

spotify_data$pop.class <- as.factor(spotify_data$pop.class) # pour la classification on reprend des facteurs

In [ ]:
head(spotify_data)

In [ ]:
X <- spotify_data[, -c(11, 14)]
y.class <- spotify_data$pop.class
y.reg <- spotify_data$popularity

In [ ]:
head(X)

In [ ]:
print(y.reg[1:15])
print(y.class[1:15])

# Apprentissage

In [ ]:
library(randomForest)
library(rpart)
library(e1071)

In [ ]:
set.seed(42)
test.ratio <- 0.25
npop <- nrow(spotify_data)
nvar <- ncol(spotify_data)
ntest <- ceiling(npop * test.ratio)
testi <- sample(1:npop, ntest)
appri <- setdiff(1:npop, testi)

## Classification

In [ ]:
# construction de l'échantillon d'apprentissage
X.train <- spotify_data[appri, -11] # on supprime popularity
# construction de l'échantillon test
X.test <- spotify_data[testi, -11]

# vérification
str(X.train)
str(X.test)
# summary(X.train)

In [ ]:
y.train.class <- y.class[appri]
y.test.class <- y.class[testi]

### Régression logistique

#### Sans pénalisation

In [ ]:
# x.mat <- model.matrix(pop.class ~ . - 1, data = X.train)
# class.lr.cv <- cv.glmnet(y = X.train$pop.class, x = x.mat, alpha = 0,
#                             lambda = c(0, 0), family = "multinomial",
#                             type.multinomial = "grouped")

# plot(class.lr.cv)

In [ ]:
paste("CV estimate of lambda :", round(class.lr.cv$lambda.1se, 3))
coef(class.lr.cv, s = "lambda.1se")

In [ ]:
pred.lr <- predict(class.lr.cv$fit, X.test[, -13], s = "lambda.min", type = "class")
table(pred.lr, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.lr, y.test.class)))/ntest), 1), " %", sep = "")

#### Pénalisation LASSO

In [ ]:
x.mat <- model.matrix(pop.class ~ . - 1, data = X.train)
class.lasso.cv <- cv.glmnet(y = X.train$pop.class, x = x.mat, alpha = 1,
                         family = "multinomial", type.multinomial = "grouped")

plot(class.lasso.cv)

In [ ]:
paste("CV estimate of lambda :", round(class.lasso.cv$lambda.1se, 3))
coef(class.lasso.cv, s = "lambda.1se")

In [ ]:
pred.lr <- predict(class.lasso.cv$glmnet.fit, newx = X.test[, -13], s = "lambda.min", type = "class")
table(pred.lr, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.lr, y.test.class)))/ntest), 1), " %", sep = "")

#### Pénalisation Ridge

In [ ]:
class.ridge.cv <- cv.glmnet(y = X.train$pop.class x = x.mat, alpha = 0,
                            family = "multinomial", type.multinomial = "grouped")

plot(class.ridge.cv)

In [ ]:
paste("CV estimate of lambda :", round(class.ridge.cv$lambda.1se, 3))
coef(class.ridge.cv, s = "lambda.1se")

In [ ]:
pred.lr <- predict(class.ridge.cv, X.test[, -13], s = "lambda.min", type = "class")
table(pred.lr, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.lr, y.test.class)))/ntest), 1), " %", sep = "")

### Random Forests

In [ ]:
set.seed(1)
eps <- 2e-02
bestMtry <- tuneRF(X.train[, -10], X.train$popularity, stepFactor = 1.5,
                   improve = eps, ntree = 500, trace = TRUE, plot = TRUE)

print(bestMtry)

In [ ]:
rf.class <- randomForest(pop.class ~ ., data = X.train, mtry = 4,
                         xtest = X.test[, -13], ytest = X.test$pop.class,
                         ntree = 500, do.trace = 50, importance = TRUE)

rf.class

In [ ]:
sort(round(importance(rf.class), 2)[, 4], decreasing = TRUE)

In [ ]:
pred.rf <- rf.class$test$predicted
table(pred.rf, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.rf, y.test.class)))/ntest), 1), " %", sep = "")

### Decision Trees

In [ ]:
tree.dis <- rpart(pop.class ~ ., data = X.train, method = "class", cp = 0.001)
plot(tree.dis)
text(tree.dis)

In [ ]:
pred.tree <- predict(tree.dis, X.test[, -13], type = "class")
table(pred.tree, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.tree, y.test.class)))/ntest), 1), " %", sep = "")

### SVC Linear Kernel

In [ ]:
svc.lin.tune <- tune.svm(pop.class ~ ., data = X.train, type = "C",
                         kernel = "lin", cross = 5,
                         cost = c(1, 1.5, 2, 2.5, 3, 3.5))
plot(svc.lin.tune)

In [ ]:
svc.lin <- svm(pop.class ~ ., data = X.train, type = "C",
               cost = svc.lin.tune$best.parameters$cost)

In [ ]:
pred.lin <- predict(svc.lin, X.test[, -13])
table(pred.lin, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.lin, y.test.class)))/ntest), 1), " %", sep = "")

### SVC Polynomial Kernel

In [ ]:
svc.poly.tune <- tune.svm(pop.class ~ ., data = X.train, type = "C",
                          kernel = "poly", cross = 5, coef0 = -1:1,
                          cost = c(1, 1.5, 2, 2.5, 3, 3.5))
# plot(svc.poly.tune)

In [ ]:
svc.poly.tune$best.parameters$coef0
svc.poly.tune$best.parameters$cost

In [ ]:
svc.poly <- svm(pop.class ~ ., data = X.train, type = "C",
                coef0 = svc.poly.tune$best.parameters$coef0,
                cost = svc.poly.tune$best.parameters$cost)

In [ ]:
pred.poly <- predict(svc.poly, X.test[, -13])
table(pred.poly, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.poly, y.test.class)))/ntest), 1), " %", sep = "")

### SVC Radial Kernel

In [ ]:
svc.rad.tune <- tune.svm(pop.class ~ ., data = X.train, type = "C",
                         kernel = "radial", cost = c(0.1, 1, 1.5, 2),
                         gamma = seq(0.02, 0.1, by = 0.02))
# plot(svc.rad.tune)

In [ ]:
svc.rad.tune$best.parameters$cost
svc.rad.tune$best.parameters$gamma

In [ ]:
svc.rad <- svm(pop.class ~ ., data = X.train, type = "C",
               cost = svc.rad.tune$best.parameters$cost,
               gamma = svc.rad.tune$best.parameters$gamma)

In [ ]:
pred.rad <- predict(svc.rad, X.test[, -13])
table(pred.rad, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.rad, y.test.class)))/ntest), 1), " %", sep = "")

### Réseaux de neurones

In [ ]:
nnet.dis <- multinom(pop.class ~ ., data = X.train, decay = 12)
nnet.dis

In [ ]:
pred.nn <- predict(nnet.dis, X.test[, -13], type = "class")
table(pred.nn, y.test.class)
paste("Prediction error: ",
      round(100 * (1 - sum(diag(table(pred.nn, y.test.class)))/ntest), 1), " %", sep = "")

## Régression

In [ ]:
X.train <- spotify_data[appri, -14] # on supprime pop.class
X.test <- spotify_data[testi, -14]

str(X.train)
str(X.test)
# summary(X.train)

In [ ]:
y.train.reg <- y.reg[appri]
y.test.reg <- y.reg[testi]

In [ ]:
plot.res <- function(x, y, titre = "titre") {
    plot(x, y, col = "blue", xlim = c(0, 100), ylim = c(-100, 100), ylab = "Résidus", 
        xlab = "Valeurs prédites", main = titre, pch = 20)
    # points(x2, y, col='red')
    abline(h = 0, col = "green")
}

In [ ]:
reg.to.class <- function(y.pred){
    n = length(y.pred)
    y.reg.to.class <- vector(mode="character", length=n)
    for (i in 1:n){
        if (0 <= y.pred[i] & y.pred[i] < 20){
            y.reg.to.class[i] = 'D'}
        else if (20 <= y.pred[i] & y.pred[i] < 40){
            y.reg.to.class[i] = 'C'}
        else if (40 <= y.pred[i] & y.pred[i] < 60){
            y.reg.to.class[i] = 'B'}
        else{
            y.reg.to.class[i] = 'A'}
    }
    return (y.reg.to.class)
}

In [ ]:
plot.regression.results <- function(y.true.reg, y.true.class, y.pred) {
    print(paste("Mean Squared Error:", round(sum((y.pred - y.true.reg)^2)/length(y.pred), 4)))
    print(paste("R2 Score:", round(cor(y.true.reg, y.pred) ^ 2, 4)))
    print(paste("Explained Variance Score:", round(1 - var(y.true.reg - y.pred)/var(y.true.reg), 4)))
    
    y.reg.to.class <- reg.to.class(y.pred)
    class.tab <- table(y.reg.to.class, y.true.class)
    print(class.tab)
    
    pred.error <- round(100 * (1 - sum(diag(table(y.reg.to.class, y.true.class)))/ntest), 1)
    print(paste("Prediction error: ", pred.error, " %", sep = ""))
    print(paste("Accuracy: ", 100 - pred.error, " %", sep = ""))
    
    preds <- data.frame(true = y.true.reg, pred = y.pred)
    
    ggplot(preds, aes(x = true, y = pred)) + geom_point(color = "blue") +
        labs(x = "true values", y = "predicted values") +
        geom_abline(intercept = 0, slope = 1, linetype = "dashed", color = "red", size = 2)
}

### Régression lineáire

#### Sans pénalisation

In [ ]:
reg.lm <- aov(popularity ~ ., data = X.train)
summary(reg.lm)

res.lm <- reg.lm$residuals
fit.lm <- reg.lm$fitted.values

plot.res(fit.lm, res.lm, "Régression linéaire sans sélection de variables")

In [ ]:
x.mat <- model.matrix(popularity ~ . - 1, data = X.train)
reg.lasso.cv <- cv.glmnet(y = X.train$popularity, x = x.mat, alpha = 1)

options(repr.plot.width = 12, repr.plot.height = 10)

plot(reg.lasso.cv)

#### Pénalisation LASSO

In [ ]:
reg.lasso.cv <- cv.glmnet(y = X.train$popularity, x = x.mat, alpha = 1)

options(repr.plot.width = 12, repr.plot.height = 10)

plot(reg.lasso.cv)

In [ ]:
# valeur estimée
paste("CV estimate of lambda :", round(reg.lasso.cv$lambda.1se, 3))

# modèle correspondant
coef(reg.lasso.cv, s = "lambda.1se")  # Extraction des valeurs ajustées et des résidus

fit.lasso <- predict(reg.lasso.cv, s = "lambda.min", newx = x.mat)
res.lasso <- X.train$popularity - fit.lasso

# Graphe des résidus
options(repr.plot.width = 16, repr.plot.height = 9)
par(mfrow = c(1, 2))

plot.res(fit.lm, res.lm, "linéaire")
plot.res(fit.lasso, res.lasso, "linéaire, pénalité L1")

#### Penalisation Ridge

In [ ]:
reg.ridge.cv <- cv.glmnet(y = X.train$popularity, x = x.mat, alpha=0)

options(repr.plot.width = 12, repr.plot.height = 10)

plot(reg.ridge.cv)

In [ ]:
paste("CV estimate of lambda :", round(reg.ridge.cv$lambda.1se, 3))
coef(reg.ridge.cv, s = "lambda.1se")

In [ ]:
fit.ridge <- predict(reg.ridge.cv, s = "lambda.min", newx = x.mat)
res.ridge <- X.train$popularity - fit.ridge

options(repr.plot.width = 16, repr.plot.height = 9)
par(mfrow = c(1, 2))
plot.res(fit.lm, res.lm, "linéaire")
plot.res(fit.ridge, res.ridge, "linéaire, pénalité L2")

#### Penalisation ElasticNet

In [ ]:
x.mat <- model.matrix(popularity ~ . - 1, data = X.train)
reg.EN.cv <- cv.glmnet(y = X.train$popularity, x = x.mat, alpha=0.5)

plot(reg.EN.cv)

In [ ]:
paste("CV estimate of lambda :", round(reg.EN.cv$lambda.1se, 3))
coef(reg.EN.cv, s = "lambda.1se")

In [ ]:
fit.EN <- predict(reg.EN.cv, s = "lambda.min", newx = x.mat)
res.EN <- X.train$popularity - fit.EN

options(repr.plot.width = 16, repr.plot.height = 9)
par(mfrow = c(1, 2))

plot.res(fit.lm, res.lm, "linéaire")
plot.res(fit.EN, res.EN, "linéaire, pénalité Elastic Net")

### Random Forest

In [ ]:
rf.reg <- randomForest(popularity ~ ., data = X.train,
                       xtest = X.test[, -10], ytest = X.test$popularity,
                       ntree = 500, do.trace = 50, importance = TRUE)

In [ ]:
fit.rfr <- rf.reg$test$predicted
res.rfr <- fit.rfr - X.test$popularity
plot.res(fit.rfr, res.rfr)

In [ ]:
set.seed(1)
eps <- 2e-05
bestMtry <- tuneRF(X.train[, -10], X.train$popularity, mtryStart = 2,
                   stepFactor = 1.5, improve = eps, ntree = 500, plot = TRUE)

bestMtry

### Decision Trees

In [ ]:
tree.reg <- rpart(popularity ~ ., data = X.train, control = rpart.control(cp = 0.001))
plot(tree.reg)
text(tree.reg)

In [ ]:
xmat <- xpred.rpart(tree.reg)
xerr <- (xmat - X.train$popularity)^2
CVerr <- apply(xerr, 2, sum)
CVerr  #    CP           erreur

# xmat_i = Y^chapeau_i est la valeur predite par le modèle qui n'a pas utilisé le
# fold conentant l'observation i xerr = (Y_i - Y_i^chapeau)^2 pour i=1,...,n

# L'erreur décroit avec la compléxité (ici ce n'est pas tout à fait le cas car
# xpred.rpart fait de la validation croisée sur l'echantillon d'apprentissage)

# En gras c'est les valeurs de gamma et à coté l'erreur estimée par validation
# croisée. On choisit le gamma avec l'erreur la plus petite.

On cherche la valeur de *cp* correspondant à la plus petite erreur.

In [ ]:
cp <- as.numeric(attributes(which.min(CVerr))$names)
cp

In [ ]:
tree.reg <- rpart(popularity ~ ., data = X.train,
                  control = rpart.control(cp = cp))

In [ ]:
library(partykit)

plot(as.party(tree.reg), type = "simple")

Graphe des résidus

In [ ]:
fit.tree <- predict(tree.reg, X.test[, -11])
res.tree <- fit.tree - X.test$popularity
plot.res(fit.tree, res.tree)

### SVR

In [ ]:
# set.seed(2021)
svm.reg.tune <- best.svm(popularity ~ ., data = X.train, cost = c(0.1, 1),
                         gamma = seq(0.02, 0.1, 0.02), cross = 5)

svm.reg.tune
# plot(svm.reg.tune)

In [ ]:
svm.reg.tune$best.parameters$cost
svm.reg.tune$best.parameters$gamma

In [ ]:
svm.reg <- svm(popularity ~ ., data = X.train,
               cost = svm.reg.tune$best.parameters$cost,
               gamma = svm.reg.tune$best.parameters$gamma)

In [ ]:
# calcul et graphe des résidus
fit.svmr <- svm.reg$fitted
res.svmr <- fit.svmr - X.train$popularity
plot.res(fit.svmr, res.svmr)

In [ ]:
pred.svr <- predict(svm.reg, newdata = X.test[, -11])
plot.regression.results(y.test.reg, y.test.class, pred.svr)

### Réseaux de neuronnes

In [ ]:
library(e1071)

nnet.tuned <- best.nnet(popularity ~ ., data = X.train, size = 4:5,
                        decay = 1:10, maxit = 200, linout = TRUE)
nnet.tuned

In [ ]:
# calcul et graphe des résidus
fit.nnetr <- nnet.tuned$fitted
res.nnetr <- fit.nnetr - X.train$popularity
plot.res(fit.nnetr, res.nnetr, titre = "")

In [ ]:
pred.nnetr <- predict(nnet.tuned, newdata = X.test[, -11])
plot.regression.results(y.test.reg, y.test.class, pred.nnetr)